In [ ]:
import os
import gc
import random
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split as TTS

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print(f'GPU found at {device_name}')
else:
  print('GPU not found')

GPU not found


In [ ]:
PATH = './drive/MyDrive/SisFall_Preprocessed/'
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
plt.rcParams['figure.figsize'] = (20,10)

SEED = 2021
def seedEverything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seedEverything()

In [ ]:
df = pd.read_pickle(os.path.join(PATH,'sensor_data.pkl'))
# for c in tqdm([f'C{i}' for i in range(9)]):
#   df[c] = df[c].apply(lambda x:eval(x)[-1999:])

In [ ]:
df.head()

,C0,C1,C2,C3,C4,C5,C6,C7,C8,filename,target
0,"[6.0, 4.0, 5.0, 3.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...","[-223.0, -224.0, -226.0, -222.0, -223.0, -224....","[149.0, 146.0, 145.0, 144.0, 144.0, 149.0, 146...","[-12.0, -12.0, -14.0, -14.0, -17.0, -18.0, -19...","[60.0, 59.0, 60.0, 61.0, 63.0, 65.0, 69.0, 70....","[-5.0, -4.0, -3.0, -3.0, -3.0, -3.0, -3.0, -2....","[46.0, 47.0, 50.0, 47.0, 49.0, 48.0, 46.0, 48....","[-878.0, -882.0, -878.0, -883.0, -883.0, -880....","[568.0, 568.0, 566.0, 569.0, 568.0, 568.0, 571...",D17_SE11_R05.txt,0
1,"[-1.0, 1.0, 1.0, -1.0, 1.0, -3.0, 1.0, -2.0, 0...","[-251.0, -252.0, -254.0, -252.0, -253.0, -254....","[-66.0, -66.0, -64.0, -64.0, -64.0, -64.0, -62...","[-18.0, -20.0, -20.0, -18.0, -15.0, -12.0, -10...","[74.0, 75.0, 75.0, 76.0, 77.0, 77.0, 78.0, 78....","[-2.0, -3.0, -3.0, -3.0, -4.0, -4.0, -4.0, -4....","[33.0, 28.0, 28.0, 27.0, 31.0, 30.0, 30.0, 32....","[-974.0, -978.0, -976.0, -984.0, -984.0, -979....","[-284.0, -284.0, -285.0, -289.0, -286.0, -283....",D16_SE11_R05.txt,0
2,"[7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 6.0, 7.0, 7.0, ...","[-222.0, -225.0, -224.0, -224.0, -224.0, -225....","[144.0, 144.0, 145.0, 146.0, 146.0, 143.0, 146...","[-14.0, -13.0, -13.0, -13.0, -12.0, -12.0, -14...","[66.0, 65.0, 63.0, 66.0, 66.0, 69.0, 69.0, 70....","[-4.0, -6.0, -6.0, -7.0, -7.0, -7.0, -8.0, -11...","[57.0, 60.0, 59.0, 59.0, 57.0, 54.0, 59.0, 58....","[-883.0, -884.0, -886.0, -881.0, -884.0, -879....","[566.0, 569.0, 568.0, 564.0, 564.0, 565.0, 567...",D17_SE11_R04.txt,0
3,"[1.0, 0.0, 0.0, 4.0, 4.0, -2.0, 3.0, 4.0, 1.0,...","[-248.0, -247.0, -246.0, -246.0, -251.0, -252....","[-75.0, -77.0, -76.0, -75.0, -74.0, -78.0, -77...","[1.0, -2.0, -4.0, -6.0, -8.0, -7.0, -6.0, -3.0...","[78.0, 79.0, 78.0, 76.0, 76.0, 75.0, 74.0, 72....","[-2.0, 0.0, -2.0, -1.0, -1.0, -2.0, -1.0, -1.0...","[28.0, 33.0, 27.0, 32.0, 35.0, 35.0, 29.0, 33....","[-957.0, -955.0, -956.0, -961.0, -959.0, -960....","[-341.0, -343.0, -341.0, -338.0, -339.0, -336....",D16_SE11_R04.txt,0
4,"[-19.0, -17.0, -16.0, -19.0, -18.0, -19.0, -18...","[-237.0, -240.0, -236.0, -239.0, -240.0, -237....","[119.0, 121.0, 132.0, 115.0, 118.0, 118.0, 118...","[-13.0, -9.0, -5.0, -4.0, -3.0, -4.0, -4.0, -1...","[72.0, 74.0, 76.0, 78.0, 78.0, 79.0, 78.0, 73....","[-5.0, -8.0, -8.0, -10.0, -10.0, -10.0, -11.0,...","[-40.0, -48.0, -46.0, -45.0, -44.0, -41.0, -32...","[-941.0, -941.0, -935.0, -937.0, -937.0, -935....","[455.0, 458.0, 452.0, 454.0, 460.0, 462.0, 453...",D17_SE11_R02.txt,0


In [ ]:
cols = [f'C{i}' for i in range(9)]
np.array(df[cols].values.tolist()).shape

(4221, 9, 1999)

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates


def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def scaled_dot_product_attention(self, q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    if mask is not None:
      scaled_attention_logits += (mask * -1e9)  
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    return output, attention_weights

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    q = self.wq(q)
    k = self.wk(k)
    v = self.wv(v)

    q = self.split_heads(q, batch_size)
    k = self.split_heads(k, batch_size)
    v = self.split_heads(v, batch_size)

    scaled_attention, attention_weights = self.scaled_dot_product_attention(q, k, v, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
    output = self.dense(concat_attention)

    return output, attention_weights


class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = self.point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def point_wise_feed_forward_network(self, d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model)
    ])

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask) 
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output) 

    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output) 

    return out2

In [ ]:
def build_sub_model(inp, name):
  pos_enc = positional_encoding(SEQ_LEN, D_MODEL)
  x_axis = tf.keras.layers.Dense(D_MODEL,
                                 name=f'{name}_x_axis', 
                                 use_bias=False)(inp[..., 0])
  y_axis = tf.keras.layers.Dense(D_MODEL, 
                                 name=f'{name}_y_axis',
                                 use_bias=False)(inp[..., 1])
  z_axis = tf.keras.layers.Dense(D_MODEL, 
                                 name=f'{name}_z_axis', 
                                 use_bias=False)(inp[..., 2])

  axis_add = tf.keras.layers.Add(name=f'{name}_axis_add')([x_axis, y_axis, z_axis])
  x = tf.keras.layers.Add(name=f'{name}_pos_enc_add')([pos_enc, axis_add])

  for _ in range(N_ENCODER_LAYER):
    x = EncoderLayer(d_model = D_MODEL,
                     num_heads = NUM_HEADS,
                     dff = D_MODEL * 4,
                     rate = LR)(x, mask=None)
  x = tf.keras.layers.GlobalAveragePooling1D()(x)
  out = tf.keras.layers.Dropout(0.25, name=f'{name}_output')(x)

  return out

def build_model():
  acc_1_inp = tf.keras.Input(shape=(SEQ_LEN, 3), name='accelaration_1_sensor_input')
  acc_2_inp = tf.keras.Input(shape=(SEQ_LEN, 3), name='accelaration_2_sensor_input')
  rot_inp = tf.keras.Input(shape=(SEQ_LEN, 3), name='rotational_sensor_input')

  acc_1_out = build_sub_model(acc_1_inp, name='accelaration_1')
  acc_2_out = build_sub_model(acc_2_inp, name='accelaration_2')
  rot_out = build_sub_model(rot_inp, name='rotational')

  x = tf.keras.layers.Add()([acc_1_out, acc_2_out, rot_out])
  out = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)
  model = tf.keras.Model([acc_1_inp, acc_2_inp, rot_inp], out)
  return model

#tf.keras.utils.plot_model(model)

In [ ]:
model = build_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 accelaration_1_sensor_input (I  [(None, 256, 3)]    0           []                               
 nputLayer)                                                                                       
                                                                                                  
 accelaration_2_sensor_input (I  [(None, 256, 3)]    0           []                               
 nputLayer)                                                                                       
                                                                                                  
 rotational_sensor_input (Input  [(None, 256, 3)]    0           []                               
 Layer)                                                                                       

In [ ]:
SEQ_LEN = 256
TYPE_1_ACC = ['C0','C1','C2']
TYPE_2_ACC = ['C6','C7','C8']
ROT_COLS = ['C3','C4','C5']
USE_COLS = TYPE_1_ACC + TYPE_2_ACC + ROT_COLS # if USE_ACC == 1 else TYPE_2_ACC + ROT_COLS

def get_tensor_data(data,idx=None):
  if idx is None:
    data = tf.constant(data[USE_COLS].values.tolist())
  else:
    data = tf.constant(data[USE_COLS].iloc[idx].values.tolist())
  data = tf.cast(data, tf.int32)
  data = tf.reshape(data, (-1, 1999, len(USE_COLS)))
  data = data[:,-SEQ_LEN:,]
  
  acc_1 = data[:,:,:3]
  acc_2 = data[:,:,3:6]
  rot = data[:,:,6:]

  return acc_1, acc_2, rot


train, test, train_label, test_label = TTS(df[USE_COLS], 
                                           df['target'], 
                                           stratify=df['target'],
                                           test_size=0.1, 
                                           random_state=SEED)
test = get_tensor_data(test)

In [ ]:
_, _, _ = get_tensor_data(df[cols])

Shape : (4221, 256, 9)


In [ ]:
D_MODEL = 24
N_ENCODER_LAYER = 6
NUM_HEADS = 4
LR = 0.01
BATCH_SIZE = 128
EPOCHS = 1
N_SPLITS = 5

skf = StratifiedKFold(n_splits=N_SPLITS)
val_score = 0
test_preds = 0

#config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
#sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
#tf.compat.v1.keras.backend.set_session(sess)

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

for i, (tr_idx, val_idx) in enumerate(skf.split(train[USE_COLS], train_label)):
  print(f'Running on fold : {i+1}')
  
  x_train = get_tensor_data(train[USE_COLS], tr_idx)
  y_train = train_label.iloc[tr_idx]

  x_val = get_tensor_data(train[USE_COLS], val_idx)
  y_val = train_label.iloc[val_idx]

  with strategy.scope():
    tf.keras.backend.clear_session()
    model = build_model()
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy', 
        metrics=[tf.keras.metrics.AUC(name='AUC')]
    )

  checkpoint_filepath = './checkpoint'
  checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                  save_weights_only=True,
                                                  monitor='val_AUC',
                                                  mode='max',
                                                  save_best_only=True,
                                                  verbose=True)

  lrSchedule = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_AUC', 
                                                    mode='max',
                                                    factor=0.025,
                                                    patience=10,
                                                    verbose=True)
  
  model.fit(x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=[checkpoint, lrSchedule],
            verbose=True)
  
  model.load_weights(checkpoint_filepath)

  eval_result = model.evaluate(x_val, y_val)
  val_score += eval_result[-1]
  test_preds += model.predict(test)

  print('='*75)

print(f'OOF_validation_score : {val_score/N_SPLITS}')
print()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.94.159.2:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.94.159.2:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on fold : 1


IndexError: ignored

In [ ]:
print(f'OOF_validation_score : {val_score/N_SPLITS}')
test_score = roc_auc_score(test_label, test_preds/N_SPLITS)
print(f'Test Score : {test_score}')

In [ ]:
"""
num_inp : 3
Epoch : 50
Seq len : 256
num_enc : 6

"""

In [ ]:
"""
num_inp : 3
Epoch : 50
Seq len : 256
num_enc : 6

Result : ResourceExhaustError

"""